# Effective number of trials

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
import ray

from functions import *

In [ ]:
FAST = True   # Set to False to have smoother / more stable plots

In [ ]:
ray.init()

# Silhouette quality

In [ ]:
# This slightly under-estimates the number of clusters

effective_number_of_trials = 10

np.random.seed(1)
for iteration in range(5):
    C, _, _ = get_random_correlation_matrix(noise=1, effective_number_of_trials=effective_number_of_trials)
    k, qualities, clusters = number_of_clusters(C)
    fig, axs = plt.subplots( 1, 2, figsize = (8,4), layout = 'constrained', dpi = 300 if iteration == 0 else 100 )
    ax = axs[0]
    ax.imshow(C, vmin = -1, vmax = +1, cmap = 'RdBu', aspect = 1, interpolation = 'nearest')
    ax.axis('off')
    ax = axs[1]
    ax.plot( qualities )
    i = np.argmax(qualities)
    x, y = qualities.index[i], qualities.iloc[i]
    ax.scatter( x, y )
    ax.text( x, y, f"  {qualities.index[i]}", va='center', ha='left' )
    ax.set_xlabel( "Number of clusters" )
    ax.set_ylabel( "Quality" )
    plt.show()

In [ ]:
@ray.remote
def f():
    C = get_random_correlation_matrix(noise=1, effective_number_of_trials = 10)[0]
    k, qualities, clusters = number_of_clusters(C)
    return k

results = [ f.remote() for _ in range(100 if FAST else 1000) ]   # 1 minute for 100 
results = [ ray.get(u) for u in tqdm(results) ]

results_1 = results

fig, ax = plt.subplots( figsize = (3,2), layout = 'constrained' )
ax.hist( 
    results, 
    bins = np.linspace( min(results) - .5, max(results) + .5, max(results) - min(results) + 2 ),
    facecolor = 'lightblue', edgecolor =  'tab:blue',
)
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xlabel( r"Estimated number of clusters" )
plt.show()        

# Effective rank

In [ ]:
# The effective rank strongly over-estimates the number of clusters
# This is sometimes what we want: if we only have 10 investment ideas, and tried a lot of noisy variants of them, 
# we have more than 10 independent trials -- the more we try, the more we are likely to find a good Sharpe ratio.

N_iterations = 10_000
effective_number_of_trials = 10
results = []
for _ in tqdm(range(N_iterations)):
    C, X, _ = get_random_correlation_matrix(noise=1, effective_number_of_trials=effective_number_of_trials)
    results.append(effective_rank(C))

results_2 = results

fig, ax = plt.subplots( figsize = (3,2), layout = 'constrained' )
ax.hist( results, facecolor = 'lightblue', edgecolor =  'tab:blue' )
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xlabel( r"Effective Number of Trials" )
plt.show()        

# Random Matrix Theory (RMT)

In [ ]:
# Check that the distribution is correct

e = []
for _ in range(1000):
    X = np.random.normal( size = X.shape )
    C = np.corrcoef( X.T )
    C = np.clip( C, -1, 1 )
    np.fill_diagonal( C, 1 )
    e.append( np.linalg.eigvalsh(C) )

e = np.hstack(e)

lambda_ = X.shape[1] / X.shape[0]
sigma = 1
lambda_plus  = sigma**2 * ( 1 + math.sqrt(lambda_) ) ** 2
lambda_minus = sigma**2 * ( 1 - math.sqrt(lambda_) ) ** 2

fig, ax = plt.subplots( figsize = (12,2), layout = 'constrained' )
ax.hist( e, bins = 200, density = True )
ax.axvline( lambda_plus, linestyle = '--', linewidth = 1, color = 'black' )
xs = np.linspace( 0, lambda_plus, 1000 )
ys = np.where(  # Marchenko-Pastur
        ( xs > lambda_minus ) & ( xs < lambda_plus ),
        1/(2*math.pi*sigma**2) * np.sqrt(
            ( lambda_plus - xs ) * ( xs - lambda_minus )
        ) / ( lambda_ * xs ),
        0,
    )
ax.plot( xs, ys, color = 'tab:orange', linewidth = 3 )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xlabel( "Distribution of the eigenvalues of the correlation matrix of N(0,I) data" )
plt.show()

In [ ]:
np.random.seed(0)
C, X, _ = get_random_correlation_matrix(noise=1, effective_number_of_trials=effective_number_of_trials)

lambda_ = X.shape[1] / X.shape[0]
sigma = 1
lambda_plus  = sigma**2 * ( 1 + math.sqrt(lambda_) ) ** 2
lambda_minus = sigma**2 * ( 1 - math.sqrt(lambda_) ) ** 2

fig, ax = plt.subplots( figsize = (12,2), layout = 'constrained' )
e = np.linalg.eigvalsh(C)
ax.hist( e, bins = 100, density = True )
ax.axvline( lambda_plus, linestyle = '--', linewidth = 1, color = 'black' )
xs = np.linspace( 0, lambda_plus, 100 )
ys = np.where(  # Marchenko-Pastur
        ( xs > lambda_minus ) & ( xs < lambda_plus ),
        1/(2*math.pi*sigma**2) * np.sqrt(
            ( lambda_plus - xs ) * ( xs - lambda_minus )
        ) / ( lambda_ * xs ),
        0,
    )
ax.plot( xs, ys, color = 'tab:orange', linewidth = 3 )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_ylabel( "Eigenvalues" )
ax.set_title( 
    "Distribution of the eigenvalues of the correlation matrix\n"
    f"Eigenvalues beyond the Marchenko-Pastur limit: {np.sum( e > lambda_plus )}"
)
plt.show()

In [ ]:
def detone_numpy(V, keep_trace=True):
    """
    Remove the largest eigenvalue of a variance matrix
    """
    trace = np.diag(V).sum()
    e, u = scipy.sparse.linalg.eigsh(V,1)
    e = e[0]
    V = V - e * u @ u.T
    if keep_trace:
        missing = trace - np.diag(V).sum()
        V += missing * np.eye(V.shape[0]) / V.shape[0]
    return V


def count_non_trivial_eigenvalues( X, naive = False ): 
    """
    Count the eigenvalues beyond the the Marchenko-Pastur limit, 
    remove them, re-compute the eigenvalues, and iterate 
    until there are no more eigenvalues beyond the limit.

    The "naive" version does this just once.

    This gives a lower bound on the number of effective trials.

    Inputs: X: numpy array, one column per trial
            naive: boolean
    Output: integer, number of non-trivial eigenvalues
    """
    C = np.corrcoef( X.T )
    C = np.clip( C, -1, 1 )
    np.fill_diagonal( C, 1 )

    lambda_ = X.shape[1] / X.shape[0]
    sigma = 1
    lambda_plus  = sigma**2 * ( 1 + math.sqrt(lambda_) ) ** 2
    lambda_minus = sigma**2 * ( 1 - math.sqrt(lambda_) ) ** 2

    count = 0
    while True: 
        e = np.linalg.eigvalsh(C)
        k = np.sum( e > lambda_plus )
        if naive or ( k == 0 ): 
            break
        count += k
        for _ in range( k ):
            C = detone_numpy(C)

    return count

In [ ]:
N_iterations = 10_000  
effective_number_of_trials = 10
results = []
for _ in tqdm(range(N_iterations)):
    C, X, _ = get_random_correlation_matrix(noise=1, effective_number_of_trials=effective_number_of_trials)
    results.append( count_non_trivial_eigenvalues(X) )
results = np.array( results ).astype(int) 
results_3 = results

fig, ax = plt.subplots( figsize = (3,2), layout = 'constrained' )
ax.hist( results, facecolor = 'lightblue', edgecolor =  'tab:blue', bins = np.linspace( -.5, max(results)+.5, max(results)+2 ) )
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xticks( np.arange( 0, 12, 2 ) )
ax.set_xlabel( "Non-trivial eigenvalues\nof the correlation matrix" )
plt.show()        

# Plots

In [ ]:
fig, axs = plt.subplots( 1, 3, figsize = (9,2), dpi = 300 )

ax = axs[0]
ax.hist( 
    results_1, 
    bins = np.linspace( min(results_1) - .5, max(results_1) + .5, max(results_1) - min(results_1) + 2 ),
    facecolor = 'lightblue', edgecolor =  'tab:blue',
)
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xticks( np.arange( 0, 14, 2 ) )
ax.set_xlabel( "Estimated number of clusters\nfrom the silhouette quality" )

ax = axs[2]
ax.hist( results_2, facecolor = 'lightblue', edgecolor =  'tab:blue' )
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xticks( np.arange( 0, 50, 10 ) )
ax.set_xlabel( "Effective rank\n of the correlation matrix" )

ax = axs[1] 
ax.hist( results_3, facecolor = 'lightblue', edgecolor =  'tab:blue', bins = np.linspace( -.5, max(results_3)+.5, max(results_3)+2 ) )
ax.axvline( effective_number_of_trials, color = 'black', linestyle = '--' )
for side in ['left', 'right', 'top']:
    ax.spines[side].set_visible(False)
ax.set_yticks([])
ax.set_xticks( np.arange( 0, 14, 2 ) )
ax.set_xlabel( "Non-trivial eigenvalues\nof the correlation matrix" )

fig.subplots_adjust(wspace=0.2)

plt.show()  